In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, Precision, Recall
from sklearn.utils.class_weight import compute_class_weight

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Moredata\\split_data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS = 30

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# 📏 Class Weight Calculation
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# 🧠 Load MobileNetV2 Base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Initially freeze base model

# 🏗️ Custom Classifier Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

# 📌 Callbacks
checkpoint = ModelCheckpoint("best_mobilenetv2_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Phase 1: Train Custom Head
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🔓 Phase 2: Unfreeze and Fine-tune MobileNetV2
base_model.trainable = True

# Re-compile with lower LR
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

# Fine-tuning Training
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🧾 Evaluate Model
loss, accuracy, auc, precision, recall = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Final Test Accuracy: {accuracy * 100:.2f}%")
print(f"📈 AUC: {auc:.4f} | 🎯 Precision: {precision:.4f} | 🔍 Recall: {recall:.4f}")



Found 226 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4598 - auc: 0.4941 - loss: 0.9436 - precision: 0.3031 - recall: 0.6061
Epoch 1: val_accuracy improved from -inf to 0.70833, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.4634 - auc: 0.5050 - loss: 0.9186 - precision: 0.2976 - recall: 0.6206 - val_accuracy: 0.7083 - val_auc: 0.7500 - val_loss: 0.5985 - val_precision: 0.4444 - val_recall: 0.6667 - learning_rate: 3.0000e-04
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6203 - auc: 0.6110 - loss: 0.7224 - precision: 0.3321 - recall: 0.5188 
Epoch 2: val_accuracy improved from 0.70833 to 0.72917, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.6263 - auc: 0.6228 - loss: 0.7116 - precision: 0.3386 - recall: 0.5258 - val_accuracy: 0.7292 - val_auc: 0.8403 - val_loss: 0.4903 - val_precision: 0.4545 - val_recall: 0.4167 - learning_rate: 3.0000e-04
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7509 - auc: 0.8105 - loss: 0.5356 - precision: 0.4904 - recall: 0.5939
Epoch 3: val_accuracy improved from 0.72917 to 0.79167, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7467 - auc: 0.8028 - loss: 0.5445 - precision: 0.4849 - recall: 0.5858 - val_accuracy: 0.7917 - val_auc: 0.8634 - val_loss: 0.4861 - val_precision: 0.5556 - val_recall: 0.8333 - learning_rate: 3.0000e-04
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7337 - auc: 0.8159 - loss: 0.5073 - precision: 0.4644 - recall: 0.7336
Epoch 4: val_accuracy improved from 0.79167 to 0.81250, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.7321 - auc: 0.8167 - loss: 0.5081 - precision: 0.4642 - recall: 0.7226 - val_accuracy: 0.8125 - val_auc: 0.8843 - val_loss: 0.4784 - val_precision: 0.5714 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7417 - auc: 0.8021 - loss: 0.5045 - precision: 0.4013 - recall: 0.7859
Epoch 5: val_accuracy did not improve from 0.81250
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7420 - auc: 0.8055 - loss: 0.5075 - precision: 0.4186 - recall: 0.7752 - val_accuracy: 0.8125 - val_auc: 0.8993 - val_loss: 0.4518 - val_precision: 0.5714 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7823 - auc: 0.8482 - loss: 0.4725 - precision: 0.5423 - recall: 0.7917 
Epoch 6: val_accuracy did not improve from 0.81250
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7789 - auc: 0.8459 - loss: 0.4753 - precision: 0.5374 - recall: 0.7905 - va